<a href="https://colab.research.google.com/github/melrepa/MCD_IngCaracteristicas/blob/main/Proyecto1_DescargandoDatos_Melissa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyecto 1: Descarga de datos

Melissa Reyes Paz

Ingeniería de características


1. Un script o libreta que descargue datos de al menos dos fuentes diferentes, y que genere un archivo texto con la descripción de las fuentes, las fechas de descarga y de ser posible la descripción (o enlaces) que expliquen la naturaleza de los datos descargados. Si los datos venían sin explicación, agregar la explicación propia para simplificar el proceso.
2. Una libreta o script que transforme y utilice los datos de acuerdo a su tipo, seleccione la información que se desea utilizar y se generen los Dataframes necesarios.
3. Un diccionario de datos por cada dataframe

Este proyecto tiene la intención de obtener información sobre la cantidad de feminicidios por estado y a nivel nacional de 2015 a 2022. 
Se localizaron dos fuentes de información en las que se encuentran los datos solicitados. 

* En los [Datos abiertos de Incidencia delictiva (SESNSP)](https://www.gob.mx/sesnsp/acciones-y-programas/datos-abiertos-de-incidencia-delictiva) se presentan los registros de las averiguaciones previas y/o carpetas de investigación de delitos y subtipo de delito  por entidad federativa y de forma mensual. En estos datos hay una sección de feminicidios los cuales se presentan registrados de forma mensual por cada estado de 2015 a septiembre de 2022.  

* En [DataMexico API Explorer](https://dev-api.datamexico.org/ui/?cube=sesnsp_crimes&cuts%5B0%5D=Type.Crime+Type%2C502&debug=false&distinct=false&drilldowns%5B0%5D=Date.Year&drilldowns%5B1%5D=Geography.State&drilldowns%5B2%5D=Type.Crime+Type&locale=es&measures%5B0%5D=Value&nonempty=true&parents=false&sparse=false) se presentan los registros de las averiguaciones previas y/o carpetas de investigación de feminicidios por entidad federativa de forma anual de 2015 a septiembre de 2022.

* En los [Datos Abiertos del Gobierno](https://datos.gob.mx/busca/dataset/estadistica-de-personas-desaparecidas-no-localizadas/resource/a42be713-6cc3-4339-88c3-0e70d1089baa?inner_span=True) se presentan las Estadísticas de Personas Desaparecidas no localizadas (EPDNL) hasta 2018. 



In [129]:
import pandas as pd
import numpy as np
import datetime
import urllib.request
import os
import requests
import csv
import json
import matplotlib.pyplot as plt
import zipfile # Descompresión de archivos

In [130]:
print(os.getcwd())
subdir = "./data/"

/content


## Accediendo a los datos por medio de URL

In [131]:
#URL datos de SESNSP
SESNSP_1522='https://drive.google.com/u/0/uc?id=1i2Zts5aDcd8cfixtA1Jn-JGlKIgYTZWN&export=download'
Diccionario_SESNSP_1522='https://drive.google.com/u/0/uc?id=1rfvgcAcEzLR1Q44wwjZhjBvBFjtGSmX3&export=download'

#Ubicacion para datos de la SESNSP
SESNSP_1522_file='IDEFC_NM_ago22.csv'
Diccionario_SESNSP_1522_file='DD_sesNSP.xlsx'

#URL datos de DataMexico API
DataMex_1522 = 'https://api.datamexico.org/tesseract/cubes/sesnsp_crimes/aggregate.jsonrecords?cuts%5B%5D=Type.Type.Crime+Type.502&debug=true&drilldowns%5B%5D=Date.Date.Year&drilldowns%5B%5D=Geography.Geography.State&measures%5B%5D=Value&parents=false&sparse=false'


#Ubicacion para datos de la API Data Mexico
DataMex_1522_file='Year-State-records.json'

#URL datos de EPDNL
EPDNL_18 = 'http://datosabiertospgr.blob.core.windows.net/desaparecidas-pgr/Estadisticade_Personas_Desaparecidas_No_Localizadas.csv'
Diccionario_EPDNL_18 = 'http://datosabiertospgr.blob.core.windows.net/desaparecidas-pgr/Diccionario_Datos_Personas_Desaparecidas_NoLocalizadas.csv'

#Ubicación para datos de EPDNL
EPDNL_18_file = 'Estadistica_Personas_Desaparecidas_No_Localizadas.csv'
Diccionario_EPDNL_18_file = 'Diccionario_Datos_Personas_Desaparecidas_NoLocalizadas.csv'


## Generando el archivo de texto con la descripción de las fuentes, las fechas de descarga y la descripción de la naturaleza de los datos descargados. 

In [132]:
SESNSP = { SESNSP_1522:SESNSP_1522_file, Diccionario_SESNSP_1522:Diccionario_SESNSP_1522_file }
DataMex = { DataMex_1522:DataMex_1522_file}
EPDNL = { EPDNL_18:EPDNL_18_file, Diccionario_EPDNL_18:Diccionario_EPDNL_18_file}

In [133]:
for url, archivo in SESNSP.items(): 
    if not os.path.exists(archivo):
        if not os.path.exists(subdir):
            os.makedirs(subdir)
        urllib.request.urlretrieve(url, subdir + archivo)  
print("Descarga de SESNSP terminada.")

for url, archivo in DataMex.items(): 
    if not os.path.exists(archivo):
        if not os.path.exists(subdir):
            os.makedirs(subdir)
        urllib.request.urlretrieve(url, subdir + archivo)
print("Descarga de DataMex terminada.")

for url, archivo in EPDNL.items(): 
    if not os.path.exists(archivo):
        if not os.path.exists(subdir):
            os.makedirs(subdir)
        urllib.request.urlretrieve(url, subdir + archivo)
print("Descarga de EPDNL terminada.")

with open(subdir + "info.txt", 'w') as f:
    f.write("Archivos con datos sobre feminicidios \n")
    info = """
    Los datos fueron descargados de los portales del Gobierno Federal y de Data México. 
    Los datos presentan información referente a delitos registrados por entidad federativa en cada año, 
    asi como tambien específicamente de la cantidad de feminicidios de 2015 a septiembre de 2022 y la estadística de las personas desaparecidas no localizadas hasta 2018. 

    """ 
    f.write(info + '\n')
    f.write("Descargado el " + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") + "\n")
    f.write("|-------------------------Metadatos de la descarga de SESNSP-------------------------|"+'\n')
    for url, archivo in SESNSP.items():        
        f.write("Desde: " + url + "\n")
        f.write("Nombre: " + archivo + "\n")
    f.write("|-------------------------Metadatos de la descarga de DataMex-------------------------|"+'\n')    
    for url, archivo in DataMex.items():        
        f.write("Desde: " + url + "\n")
        f.write("Nombre: " + archivo + "\n")
    f.write("|-------------------------Metadatos de la descarga de EPDNL-------------------------|"+'\n')
    for url, archivo in EPDNL.items():        
        f.write("Desde: " + url + "\n")
        f.write("Nombre: " + archivo + "\n")

Descarga de SESNSP terminada.
Descarga de DataMex terminada.
Descarga de EPDNL terminada.


## Tidy DataFrame a partir de los datos del Secretariado Ejecutivo del Sistema Nacional de Seguridad Pública (SESNSP)

La tabla de datos descargados presenta un registro de la cantidad y tipo de delitos registrados cada mes de 2015 a 2022 por entidad federativa.  
Incluye tipo de delito, el subtipo, la modalidad. Sin embargo, lo que se está buscando es la información referente a los feminicidios, por lo que se tuvo la libertad de prescindir la información referente al resto de los delitos. 
Otra cuestión fue que el numero de delitos viene registrado de forma mensual, y al querer trabajar con datos anuales, se optó por hace una sumatoria para obtener el total anual. Para hacer esta suma, se descartó la información de año y Clave_Ent ya que son valores Int y podían interferir con el resultado de la suma. 

### Limpieza.

In [134]:
sesnsp=pd.read_csv("data//"+SESNSP_1522_file, encoding='latin-1')

In [135]:
sesnsp_columns=['Año', 
                'Clave_Ent', 
                'Entidad', 
                'Bien jurídico afectado',
                'Tipo de delito',
                'Subtipo de delito',
                'Modalidad',
                'Enero',  
                'Febrero',
                'Marzo',
                'Abril',
                'Mayo',
                'Junio',
                'Julio',
                'Agosto',
                'Septiembre',
                'Octubre',
                'Noviembre',
                'Diciembre']


SESNSP_tidy=sesnsp[sesnsp_columns].copy()

#Creando lista de columna para descartar Año y Clave_Ent para el momento de la sumatoria. 
col_listSESNSP= list(SESNSP_tidy)
col_listSESNSP.remove('Año')
col_listSESNSP.remove('Clave_Ent')
#Suma de columnas numéricas (solo los datos registrados en meses para obtener el total anual)
SESNSP_tidy['Total'] = SESNSP_tidy[col_listSESNSP].sum(axis=1)

#Descartando columnas que no se usarán
SESNSP_tidy.drop(['Clave_Ent', 
                'Bien jurídico afectado', 
                'Subtipo de delito', 
                'Modalidad',
                'Enero',  
                'Febrero',
                'Marzo',
                'Abril',
                'Mayo',
                'Junio',
                'Julio',
                'Agosto',
                'Septiembre',
                'Octubre',
                'Noviembre',
                'Diciembre'], axis = 'columns', inplace=True)

#Descartando filas que no pertenecen al delito Feminicidio
SESNSP_tidy = SESNSP_tidy.drop(SESNSP_tidy[SESNSP_tidy['Tipo de delito']!='Feminicidio'].index)

#Descartando filas que no pertenecen a los años 2015-2018
SESNSP_tidy = SESNSP_tidy.drop(SESNSP_tidy[SESNSP_tidy['Año']> 2018].index)

#Renombrando columna
SESNSP_tidy = SESNSP_tidy.rename(columns ={'Tipo de delito':'Delito'})

#Convirtiendo valores float a int
convert_dict = {'Total': int}
SESNSP_tidy = SESNSP_tidy.astype(convert_dict)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.


In [136]:
#Uniendo filas pertenecientes al mismo año y entidad.
def unir_feminicidios(SESNSP_tidy):
  f1 = ", ".join(f"{fem:}" for fem in set(SESNSP_tidy.Delito.dropna()))
  t = np.sum(SESNSP_tidy.Total.dropna())
  return pd.DataFrame({"Delito":[f1], "Total":[t]})

SESNSP_tidy = SESNSP_tidy.groupby(["Año", "Entidad"]).apply(unir_feminicidios).droplevel(-1).reset_index()

#Ordenando por estado
SESNSP_tidySt = SESNSP_tidy.sort_values("Entidad")

### Tabla.
Primero se puede observar el dataframe con los datos de cada estado por cada año(primero todo 2015, seguido de todo 2016, etc.) Despues se observa el dataframe con los datos de cada año por cada estado (primero todo lo referente a Aguascalientes, seguido de Baja California, etc.)

In [137]:
SESNSP_tidy

,Año,Entidad,Delito,Total
0,2015,Aguascalientes,Feminicidio,0
1,2015,Baja California,Feminicidio,10
2,2015,Baja California Sur,Feminicidio,0
3,2015,Campeche,Feminicidio,4
4,2015,Chiapas,Feminicidio,36
...,...,...,...,...
123,2018,Tamaulipas,Feminicidio,13
124,2018,Tlaxcala,Feminicidio,4
125,2018,Veracruz de Ignacio de la Llave,Feminicidio,101
126,2018,Yucatán,Feminicidio,7


In [138]:
SESNSP_tidySt

,Año,Entidad,Delito,Total
0,2015,Aguascalientes,Feminicidio,0
32,2016,Aguascalientes,Feminicidio,0
64,2017,Aguascalientes,Feminicidio,0
96,2018,Aguascalientes,Feminicidio,4
33,2016,Baja California,Feminicidio,12
...,...,...,...,...
30,2015,Yucatán,Feminicidio,1
63,2016,Zacatecas,Feminicidio,11
31,2015,Zacatecas,Feminicidio,0
95,2017,Zacatecas,Feminicidio,11


### Diccionario.

#### Datos originales 

| Variable             | Etiqueta                                       | Descripción de la variable                                                               | Tipo de variable     |
|----------------------|------------------------------------------------|-----------------------------------------------------------------------------------------|----------------------|
| Año                 | Año                                         | Año de registro de las averiguaciones previas y/o carpetas de investigación.                                                  | Int               |
| Clave_Ent                  | Clave de la entidad                                           | Clave de la entidad, según el Marco Geoestadístico Nacional (MGN) del Instituto Nacional de Geografía y Estadística (INEGI).                                                    | Int  |
| Entidad                  | Entidad                                           | Entidad federativa de registro de las averiguaciones previas y/o carpetas de investigación.                                                    | String                  |
| Bien jurídico afectado       | Bien jurídico afectado                               | Primera clasificación de los delitos en las averiguaciones previas y/o carpetas de investigación.                                      | String             |
| Tipo de delito       | Tipo de delito                           | Segunda clasificación de los delitos.                                           | String               |
| Subtipo de delito   | Subtipo de delito | Tercera clasificación de los delitos. | String               |
| Modalidad       | Modalidad                         | Cuarta clasificación de los delitos.                          | String               |
| Enero - Diciembre | Meses de Enero a Diciembre                | Mes de registro de las averiguaciones previas y/o carpetas de investigación. Se incluye el número de registros por mes.                    | Float               |


#### Datos tidy

| Variable| Etiqueta| Descripción de la variable| Tipo de variable|
|---------|---------|---------------------------|-----------------|
| Año| Año| Año de registro de las averiguaciones previas y/o carpetas de investigación. | Int|
| Estado| Entidad/Estado| Entidad federativa de registro de las averiguaciones previas y/o carpetas de investigación.| String|
|Delito| Tipo de delito| Feminicidios| String 
| Total | Valor/Cantidad| Número de feminicidios registrados/denunciados.| Int|


## Tidy DataFrame a partir de los datos de la API de la Plataforma DataMexico.

La tabla de datos descargados presenta un registro de la cantidad de feminicidios registrados de 2015 a 2022 por entidad federativa.  
Esta API cuenta con botones de descarga para obtener los archivos en CSV, JSON tidy y JSON array, y tambien dos enlaces de descarga que proporciona un archivo json anidado.

### Limpieza.
De este lado del código, nunca habíamos trabajado con archivos tipo json con una estructura, que al parecer se le llama "nested".
Es por ello que al querer leer el archivo como json, nos decía que no se podía leer por no tener márgenes iguales.

El proceso de limpieza fue como a continuación.

In [139]:
datamex = "data//" + DataMex_1522_file
with open(datamex, encoding="utf-8") as f:
  datam = json.load(f)

#Normalizando datos json
  dm = pd.json_normalize(datam)
  dm_exploded = dm.apply(lambda x: x.explode()).reset_index(drop = True)

#Comprobando, en función de la primera fila, si es de tipo dict
columns_dict = dm_exploded.columns[dm_exploded.apply(lambda x: isinstance(x[0], dict))]

#Agregar el dict dividido al DataFrame
for col in columns_dict:
  dm_splitted_dict = dm_exploded[col].apply(pd.Series)
  dm_exploded = pd.concat([dm_exploded, dm_splitted_dict], axis = 1)

In [140]:
datamex_columns=['data', 
                'source', 
                'Year', 
                'State ID',
                'State',
                'Value',
                'name',
                'measures',  
                'annotations']


DataMex_tidy=dm_exploded[datamex_columns].copy()

#Elimindando columnas que no se utilizarán
DataMex_tidy.drop(['data', 
                'source', 
                'State ID', 
                'name', 
                'measures',  
                'annotations'], axis = 'columns', inplace=True)


#Descartando filas que no pertenecen a los años 2015-2018
DataMex_tidy = DataMex_tidy.drop(DataMex_tidy[DataMex_tidy['Year']> 2018].index)

#Ordenando por estado
DataMex_tidySt = DataMex_tidy.sort_values("State")

### Tabla.

Primero de puede observar el dataframe con los datos de cada estado por cada año(primero todo 2015, seguido de todo 2016, etc.)
Despues se observa el dataframe con los datos de cada año por cada estado (primero todo lo referente a Aguascalientes, seguido de Baja California, etc.)

In [141]:
DataMex_tidy

,Year,State,Value
0,2015,Aguascalientes,0
1,2015,Baja California,10
2,2015,Baja California Sur,0
3,2015,Campeche,4
4,2015,Coahuila de Zaragoza,16
...,...,...,...
123,2018,Tamaulipas,13
124,2018,Tlaxcala,4
125,2018,Veracruz de Ignacio de la Llave,101
126,2018,Yucatán,7


In [142]:
DataMex_tidySt

,Year,State,Value
0,2015,Aguascalientes,0
32,2016,Aguascalientes,0
64,2017,Aguascalientes,0
96,2018,Aguascalientes,4
65,2017,Baja California,11
...,...,...,...
30,2015,Yucatán,1
63,2016,Zacatecas,11
31,2015,Zacatecas,0
95,2017,Zacatecas,11


### Diccionario.

#### Datos originales

| Variable| Etiqueta| Descripción de la variable| Tipo de variable|
|---------|---------|---------------------------|-----------------|
| data | data | Toda la información de la fila. | String |
| source | source | Fuente de la información | String |
| Year| Año| Año de registro de las averiguaciones previas y/o carpetas de investigación. | Int|
| State ID| Clave de la entidad| Clave de la entidad, según el Marco Geoestadístico Nacional (MGN) del Instituto Nacional de Geografía y Estadística (INEGI). | Int  |
| State| Entidad/Estado| Entidad federativa de registro de las averiguaciones previas y/o carpetas de investigación.| String|
| Value | Valor/Cantidad| Número de feminicidios registrados/denunciados.| Int|
| name | name | Nombre de la base de datos. | String |
| measures | measures | Valor | String |
| annotations | annotations | Información sobre la fuente. | String |


#### Datos tidy

| Variable| Etiqueta| Descripción de la variable| Tipo de variable|
|---------|---------|---------------------------|-----------------|
| Year| Año| Año de registro de las averiguaciones previas y/o carpetas de investigación. | Int|
| State| Entidad/Estado| Entidad federativa de registro de las averiguaciones previas y/o carpetas de investigación.| String|
| Value | Valor/Cantidad| Número de feminicidios registrados/denunciados.| Int|


## Tidy DataFrame a partir de los datos de las Estadísticas de personas desaparecidas-no localizadas (EPDNL).

La tabla de datos descargados pertenece a las Estadísticas de personas desaparecidas-no localizadas. Incluye las denuncias por estado de forma mensual y está dividido en varias categorías como reportados como desaparecidos, encontrados con o sin vida, etc. La información va desde 2014 hasta 2018.

### Limpieza.
La información original incluye las denuncias de personas desaparecidas divididos en hombres y mujeres. Solo se conservará la información relacionada con mujeres para después ser comparada con la cantidad de feminicidios denunciados.

In [156]:
epdnl=pd.read_csv("data//"+EPDNL_18_file, encoding='latin-1')
dicepdnl=pd.read_csv("data//"+Diccionario_EPDNL_18_file, encoding = 'latin-1')

In [157]:
epdnl_columns=['AÑO',
               'MES',
               'Entidad donde se recibió la Denuncia',
                'RD_H', 
                'RD_M', 
                'RD_215_H', 
                'RD_215_M',
                'EVDI_H',
                'EVDI_M',
                'EVDV_H',
                'EVDV_M',  
                'EV_215_I_H',
                'EV_215_I_M',
                'EV_215_V_H',
                'EV_215_V_M',
                'DESV_H',
                'DESV_M',
                'DESV_215_H',
                'DESV_215_M',
                'CDNB_H',
                'CDNB_M',
                'CDNB_215_H',
                'CDNB_215_M']


#Descartando columnas que no se usarán
epdnl.drop(['MES',
            'RD_H', 
            'RD_215_H', 
            'EVDI_H',
            'EVDV_H',
            'EV_215_I_H',
            'EV_215_V_H',
            'DESV_H',
            'DESV_215_H',
            'CDNB_H',
            'CDNB_215_H'], axis = 'columns', inplace=True)
EPDNL_tidy = epdnl


#Descartando filas que no pertenecen a los años 2015-2018
EPDNL_tidy = EPDNL_tidy.drop(EPDNL_tidy[EPDNL_tidy['AÑO']< 2015].index)

#Renombrando columna
EPDNL_tidy = EPDNL_tidy.rename(columns ={'Entidad donde se recibió la Denuncia':'Estado'})

In [158]:

#Uniendo filas pertenecientes al mismo año y entidad.
def unir_desap(EPDNL_tidy):
  R = np.sum(EPDNL_tidy.RD_M.dropna())
  R215 = np.sum(EPDNL_tidy.RD_215_M.dropna())
  EVDI = np.sum(EPDNL_tidy.EVDI_M.dropna())
  EVDV = np.sum(EPDNL_tidy.EVDV_M.dropna())
  EV = np.sum(EPDNL_tidy.EV_215_I_M.dropna())
  EVV = np.sum(EPDNL_tidy.EV_215_V_M.dropna())
  DESV = np.sum(EPDNL_tidy.DESV_M.dropna())
  DESV215 = np.sum(EPDNL_tidy.DESV_215_M.dropna())
  CDNB = np.sum(EPDNL_tidy.CDNB_M.dropna())
  CDNB215 = np.sum(EPDNL_tidy.CDNB_215_M.dropna())
  return pd.DataFrame({"RD_M":[R], "RD_215_M":[R215], "EVDI_M":[EVDI], "EVDV_M":[EVDV], "EV_215_I_M":[EV], "EV_215_V_M":[EVV],
                       "DESV_M":[DESV], "DESV_215_M":[DESV215], "CDNB_M":[CDNB], "CDNB_215_M":[CDNB215]})

EPDNL_tidy = EPDNL_tidy.groupby(["AÑO", "Estado"]).apply(unir_desap).droplevel(-1).reset_index()



In [159]:
#Ordenando por estado
EPDNL_tidySt = EPDNL_tidy.sort_values("Estado")

### Tabla.
Primero de puede observar el dataframe con los datos de cada estado por cada año(primero todo 2015, seguido de todo 2016, etc.)
Despues se observa el dataframe con los datos de cada año por cada estado (primero todo lo referente a Aguascalientes, seguido de Baja California, etc.)

In [160]:
EPDNL_tidy

,AÑO,Estado,RD_M,RD_215_M,EVDI_M,EVDV_M,EV_215_I_M,EV_215_V_M,DESV_M,DESV_215_M,CDNB_M,CDNB_215_M
0,2015,AGUASCALIENTES,0,0,0,0,0,0,0,0,0,0
1,2015,BAJA CALIFORNIA,4,0,0,0,0,0,0,0,4,0
2,2015,BAJA CALIFORNIA SUR,0,0,0,0,0,0,0,0,0,0
3,2015,CAMPECHE,0,0,0,0,0,0,0,0,0,0
4,2015,CHIAPAS,1,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
123,2018,TAMAULIPAS,1,2,0,0,0,0,0,0,1,0
124,2018,TLAXCALA,0,1,0,0,1,0,0,0,0,1
125,2018,VERACRUZ DE IGNACIO DE LA LLAVE,0,1,0,0,1,0,0,0,0,1
126,2018,YUCATÁN,0,0,0,0,0,0,0,0,0,0


In [161]:
EPDNL_tidySt

,AÑO,Estado,RD_M,RD_215_M,EVDI_M,EVDV_M,EV_215_I_M,EV_215_V_M,DESV_M,DESV_215_M,CDNB_M,CDNB_215_M
0,2015,AGUASCALIENTES,0,0,0,0,0,0,0,0,0,0
32,2016,AGUASCALIENTES,0,0,0,0,0,0,0,0,0,0
64,2017,AGUASCALIENTES,0,0,0,0,0,0,0,0,0,0
96,2018,AGUASCALIENTES,0,0,0,0,0,0,0,0,0,0
33,2016,BAJA CALIFORNIA,1,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
30,2015,YUCATÁN,0,0,0,0,0,0,0,0,0,0
63,2016,ZACATECAS,0,0,0,0,0,0,0,0,0,0
31,2015,ZACATECAS,0,0,0,0,0,0,0,0,0,0
95,2017,ZACATECAS,0,0,0,0,0,0,0,0,0,0


### Diccionario.

#### Datos originales 

| Variable             | Etiqueta                                       | Descripción de la variable                                                               | Tipo de variable     |
|----------------------|------------------------------------------------|-----------------------------------------------------------------------------------------|----------------------|
| AÑO | Año | Año en que se recibió la denuncia. | Int |
| MES | Mes | Mes en que se recibió la denuncia. | String |
| Entidad donde se realizó la denuncia. | Entidad | Entidad donde se realizó la denuncia. | String |
| RD_H                 | RD_H                                         | Reportados como desaparecidos Hombres                                                  | Int               |
| RD_M                  | RD_M                                           | Reportados como desaparecidos Mujeres                                                    | Int  |
| RD_215_H                  | RD_215_H                                           |  Reportados como desaparecidos por Artículo 215 A-D del CPF_Hombres                                                  | Int                  |
| RD_215_M       | RD_215_M                               |  Reportados como desaparecidos por Artículo 215 A-D del CPF_Mujeres                                     | Int             |
| EVDI_H       | EVDI_H                           | Encontrados vivos desaparecidos involuntariamente_Hombres                                           | Int               |
| EVDI_M   | EVDI_M | Encontrados vivos desaparecidos involuntariamente_Mujeres | Int               |
| EVDV_H       | EVDV_H                         | Encontrados vivos desaparecidos voluntariamente_Hombre                          | Int               |
| EVDV_M | EVDV_M                |  Encontrados vivos desaparecidos voluntariamente_Mujeres                   | Int               |
| EV_215_I_H	| EV_215_I_H	| Encontrados vivos por Artículo 215 A-D del CPF involuntariamente_Hombres | Int |
| EV_215_I_M | EV_215_I_M | Encontrados vivos por Artículo 215 A-D del CPF involuntariamente_Mujeres | Int |
| EV_215_V_H | EV_215_V_H | Encontrados vivos por Artículo 215 A-D del CPF voluntariamente_Hombres | Int |
| EV_215_V_M | EV_215_V_M | Encontrados vivos por Artículo 215 A-D del CPF voluntariamente_Mujeres | Int |
| DESV_H | DESV_H | Desaparecidos encontrados sin vida Hombres | Int |
| DESV_M | DESV_M | Desaparecidos encontrados sin vida Mujeres | Int |
| DESV_215_H | DESV_215_H | Desaparecidos encontrados sin vida por Artículo 215 A-D del CPF_Hombre | Int |
| DESV_215_M | DESV_215_M | Desaparecidos encontrados sin vida por Artículo 215 A-D del CPF_Mujeres | Int |
| CDNB_H | CDNB_H | Cantidad de desaparecidos que ya no se buscan Hombres | Int |
| CDNB_M | CDNB_M | Cantidad de desaparecidos que ya no se buscan Mujeres  | Int |
| CDNB_215_H | CDNB_215_H | Cantidad de personas desaparecidas por Artículo 215 A-D del CPF que ya no se buscan_Hombres | Int |
| CDNB_215_M | CDNB_215_M | Cantidad de personas desaparecidas por Artículo 215 A-D del CPF que ya no se buscan_Mujeres | Int |

#### Datos tidy 

| Variable             | Etiqueta                                       | Descripción de la variable                                                               | Tipo de variable     |
|----------------------|------------------------------------------------|-----------------------------------------------------------------------------------------|----------------------|
| AÑO | Año | Año en que se recibió la denuncia. | Int |
| Entidad donde se realizó la denuncia. | Entidad | Entidad donde se realizó la denuncia. | String |
| RD_M                  | RD_M                                           | Reportados como desaparecidos Mujeres                                                    | Int  |
| RD_215_M       | RD_215_M                               |  Reportados como desaparecidos por Artículo 215 A-D del CPF_Mujeres                                     | Int             |
| EVDI_M   | EVDI_M | Encontrados vivos desaparecidos involuntariamente_Mujeres | Int               |
| EVDV_M | EVDV_M                |  Encontrados vivos desaparecidos voluntariamente_Mujeres                   | Int               |
| EV_215_I_M | EV_215_I_M | Encontrados vivos por Artículo 215 A-D del CPF involuntariamente_Mujeres | Int |
| EV_215_V_M | EV_215_V_M | Encontrados vivos por Artículo 215 A-D del CPF voluntariamente_Mujeres | Int |
| DESV_M | DESV_M | Desaparecidos encontrados sin vida Mujeres | Int |
| DESV_215_M | DESV_215_M | Desaparecidos encontrados sin vida por Artículo 215 A-D del CPF_Mujeres | Int |
| CDNB_M | CDNB_M | Cantidad de desaparecidos que ya no se buscan Mujeres  | Int |
| CDNB_215_M | CDNB_215_M | Cantidad de personas desaparecidas por Artículo 215 A-D del CPF que ya no se buscan_Mujeres | Int |

## Comparando datos.

Considerando la naturaleza de los datos, principalmente porque no se contaba con información sobre personas desaparecidas después de 2018, se optó por trabajar con los datos de forma anual y por estado desde 2015 hasta 2018. 
* Se tienen dos dataframes con la misma información, cantidad de feminicidios denunciados por año en cada estado, pero el origen de la información es distinta, al igual que el formato con el que se trabajó. 
* Se trabajó con las dos fuentes hasta obtener dataframes iguales. 
* Se eligió uno de los dos (SESNSP) para ser comparado con la información de EPDNL. 

In [164]:
result = pd.concat([SESNSP_tidySt, EPDNL_tidySt], axis=1)
#Descartando columnas que no se usarán
result.drop(['Delito',
            'AÑO', 
            'Estado'], axis = 'columns', inplace=True)
Result = result


In [166]:
Result

,Año,Entidad,Total,RD_M,RD_215_M,EVDI_M,EVDV_M,EV_215_I_M,EV_215_V_M,DESV_M,DESV_215_M,CDNB_M,CDNB_215_M
0,2015,Aguascalientes,0,0,0,0,0,0,0,0,0,0,0
32,2016,Aguascalientes,0,0,0,0,0,0,0,0,0,0,0
64,2017,Aguascalientes,0,0,0,0,0,0,0,0,0,0,0
96,2018,Aguascalientes,4,0,0,0,0,0,0,0,0,0,0
33,2016,Baja California,12,1,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
30,2015,Yucatán,1,0,0,0,0,0,0,0,0,0,0
63,2016,Zacatecas,11,0,0,0,0,0,0,0,0,0,0
31,2015,Zacatecas,0,0,0,0,0,0,0,0,0,0,0
95,2017,Zacatecas,11,0,0,0,0,0,0,0,0,0,0


### Diccionario


#### Datos tidy 

| Variable             | Etiqueta                                       | Descripción de la variable                                                               | Tipo de variable     |
|----------------------|------------------------------------------------|-----------------------------------------------------------------------------------------|----------------------|
| Año | Año | Año en que se recibió la denuncia de feminicidio y personas desaparecidas. | Int |
| Entidad donde se realizó la denuncia. | Entidad | Entidad donde se realizó la denuncia. | String |
| Total | Valor/Cantidad| Número de feminicidios registrados/denunciados.| Int|
| RD_M                  | RD_M                                           | Reportados como desaparecidos Mujeres                                                    | Int  |
| RD_215_M       | RD_215_M                               |  Reportados como desaparecidos por Artículo 215 A-D del CPF_Mujeres                                     | Int             |
| EVDI_M   | EVDI_M | Encontrados vivos desaparecidos involuntariamente_Mujeres | Int               |
| EVDV_M | EVDV_M                |  Encontrados vivos desaparecidos voluntariamente_Mujeres                   | Int               |
| EV_215_I_M | EV_215_I_M | Encontrados vivos por Artículo 215 A-D del CPF involuntariamente_Mujeres | Int |
| EV_215_V_M | EV_215_V_M | Encontrados vivos por Artículo 215 A-D del CPF voluntariamente_Mujeres | Int |
| DESV_M | DESV_M | Desaparecidos encontrados sin vida Mujeres | Int |
| DESV_215_M | DESV_215_M | Desaparecidos encontrados sin vida por Artículo 215 A-D del CPF_Mujeres | Int |
| CDNB_M | CDNB_M | Cantidad de desaparecidos que ya no se buscan Mujeres  | Int |
| CDNB_215_M | CDNB_215_M | Cantidad de personas desaparecidas por Artículo 215 A-D del CPF que ya no se buscan_Mujeres | Int |